# Students Do: Sentiment Analysis - RNNs Vs. Vader

In this activity, students will use two different models to score sentiment. The goal is to put the performance metrics and techniques students have learned into action to decide which model performs better between VADER and RNN LSTM.

In [2]:
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf

%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
C:\Users\nospm\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [10]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
# Set the random seed for reproducibility
from numpy.random import seed
seed(1)

from tensorflow import random
random.set_seed(2)

## The DataSet: IMBD Movie Reviews

The dataset provided contains `25000` movie reviews based on [the data shared by Andrew Mass from Stanford University](http://ai.stanford.edu/~amaas/data/sentiment/). This dataset is intended to serve as a benchmark for sentiment classification, that's why it's suitable for this activity.

The movie reviews are split evenly into `12500` positive reviews and `12500` negative reviews. The reviews are not attached to a particular movie, and this is not crucial for our models' comparison since we would like to benchmark which model performs better to score sentiment.

You can learn more about this dataset in the following research paper: [Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng and Christopher Potts. (2011). **Learning Word Vectors for Sentiment Analysis**. The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011)](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf).

## Instructions

### Data Preprocessing

Load the provided dataset in a Pandas DataFrame entitled `df_reviews` and show the first `10` records.

In [6]:
# Load the dataset
file_path = Path("./Resources/movie_comments.csv")
df_reviews = pd.read_csv(file_path)
df_reviews.head()

,comment,sentiment
0,A question for all you girls out there : If a ...,0
1,It was almost worth sitting through this entir...,0
2,One of the weaker Carry On adventures sees Sid...,0
3,"First of all, I think the below comment is unw...",1
4,This is by far the worst film I have seen in m...,0


Create the features set `X` and the target vector `y` by assigning the `comment` column to `X` and the `sentiment` column to `y`.

In [7]:
# Create the features set (X) and the target vector (y)
X = df_reviews["comment"].values
y = df_reviews["sentiment"].values

Use the `train_test_split` method from `sklearn` to create the training, testing, and validation sets.

In [8]:
# Create the train, test, and validation sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=78)

## Scoring Sentiment Using VADER

In this section, you will use VADER sentiment from the `nltk` library to score the sentiment of the testing set. Later, you will assess model performance using metrics such as accuracy, precision, recall, among others.

In [ ]:
# Import the libraries for sentiment scoring using Vader
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

Start by downloading or updating the VADER lexicon.

In [11]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\nospm\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

Create an instance of the `SentimentIntensityAnalyzer` and name it `analyzer`.

In [12]:
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

Define two lists to store the sentiment scoring results as follows:

* `y_vader_pred` will save the scored sentiment, `1` for positive or `0` for negative.

* `y_vader_prob` will store the normalized value of the `pos` polarity score.

In [17]:
# Define two lists to store vader sentiment scoring
y_vader_pred = []
y_vader_prob = []

Create a `for` loop to iterate across all the comments in the `X` set and score the sentiment of each review comment. Update the two lists for sentiment scores as follows:

* Append the `pos` score to the `y_vader_prob`, you will normalize this list's values later.

* To score a review comment as positive or negative, we will use the `compound` polarity score; as you may remember from the NLP unit, the `compound` score ranges between `-1` (most extreme negative) and `+1` (most extreme positive). Following the recommendations from [this research paper](https://scholar.smu.edu/cgi/viewcontent.cgi?article=1051&context=datasciencereview), we will define a threshold of `0.1` to label a review as positive, if the `compound` score is greater than or equal to `0.1`, the review comment will be positive (append `1` to `y_vader_pred`); if the `compound` score is below `0.1`, the review comment will be negative (append `0` to `y_vader_pred`).

In [15]:
X_test[0]

'I heard so much about this movie how it was a great slasher and one of those early 80\'s movies that die hard fans of most slasher movies just had to see. Well, I rented it and I have to say that although it kept my attention as far as the suspense goes for most slasher films such as "April Fools Day", "Friday 13th" and "Prom Night", this film could have been right up there with the above mentioned only it lacked true enthusiasm and potential from the characters as well as the on going story. Characters that I found were unfortunate to be in this movie was the weirdo guy with the frizzy hair that kept creeping around the dorm and of course leading up to his true climatic role during the end with he faces the killer. Another would be the dirty scruffy looking guy with the jean jacket, he could have played more roles in this movie that might have made the movie more interesting, instead, the movie played this guy as just another loser out there making unknown calls while he sleeps with 

In [18]:
# Score sentiment of test set using Vader
for comment in X_test:

        sentiment = analyzer.polarity_scores(comment)        
        
        pos = sentiment["pos"]
        y_vader_prob.append(pos)

        # Between -1 and 1
        compound = sentiment["compound"]
        if compound > 0.1:
            y_vader_pred.append(1)
        else:
            y_vader_pred.append(0)  

In [19]:
print(f'{len(y_vader_pred)} and {len(y_vader_pred)}')

6250 and 6250


You will use the values from the `pos` polarity score to plot the ROC curve; we will consider the `pos` score as the probability of a review comment to be positive. To plot the ROC curve, these probabilities should range from `0` to `1`, so the values of the `y_vader_prob` list will be normalized using [min-max normalization](https://en.wikipedia.org/wiki/Feature_scaling#Rescaling_(min-max_normalization)).

* Normalize the data stored in the `y_vader_prob` list and save the resulting normalized values in a variable called `y_vader_prob_norm`.

_Hint:_ To normalize the data, you can use the `MinMaxScaler` method from `sklearn`, or you can code the min-max normalization formula using a comprehension list.

In [25]:
type(y_vader_prob)

list

In [30]:
y_vader_prob[0]

0.095

In [35]:
# Option 1: Normalizing data using MinMaxScaler from sklearn
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(np.array(y_vader_prob).reshape(-1,1))
y_vader_prob_norm = scaler.transform(np.array(y_vader_prob).reshape(-1,1))

In [37]:
y_vader_prob_norm[:5]

array([[0.18164436],
       [0.66730402],
       [0.11472275],
       [0.2791587 ],
       [0.49139579]])

In [ ]:
# Option 2: Using a comprehension list


## Scoring Sentiment Using RNN LSTM

In this section, you will build an RNN LSTM model to score the sentiment of the review comments. You will fit the model using the training and validations sets, and finally, you will get some predictions using the testing set for further performance assessment and comparison with VADER.

Start encoding the review comments using the `Tokenizer` method from Keras.

In [39]:
# Import the Tokenizer method from Keras
from tensorflow.keras.preprocessing.text import Tokenizer

Create an instance of the `Tokenizer`, and fit it with all the review comments that you stored in `X`.

In [42]:
# Create an instance of the Tokenizer and fit it with the X text data
tokenizer = Tokenizer(lower=True)
tokenizer.fit_on_texts(X_train)

For testing proposes, print the first five elements of the encoded vocabulary created with the `Tokenizer`.

In [43]:
# Print the first five elements of the encoded vocabulary
for token in list(tokenizer.word_index)[:5]:
    print(f"word: '{token}', token: {tokenizer.word_index[token]}")

word: 'the', token: 1
word: 'and', token: 2
word: 'a', token: 3
word: 'of', token: 4
word: 'to', token: 5


To fit the RNN LSTM model for sentiment scoring, the text data in `X` should be encoded as sequences. Use the `text_to_sequence()` method of the `tokenizer` to transform the text data to numerical sequences and save the sequences in a variable called `X_seq`.

In [46]:
# Transform the text data to numerical sequences
X_seq = tokenizer.texts_to_sequences(X)

For testing proposes, compare the text representation of a movie review with its numerical representation, by printing the first text review in `X` and the first encoded element in `X_seq`.

In [47]:
# Contrast a sample numerical sequence with its text version
print('Text: {x[0]}')

Text: {x[0]}


In [48]:
print('Token numeric: {X_seq[0]}')

Token numeric: {X_seq[0]}


Recall that RNN LSTM models need equal size inputs, so that, pad the sequences stored in `X_pad` up to `140` integers using the `pad_sequences` method from Keras. Store the pad size in a variable called `max_words`.

**Note:** You may use a bigger padding size; however, using a bigger value will increase the time that takes fitting the RNN LSTM model.

In [ ]:
# Import the pad_sequences method from Keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Set the pad size
max_words = 

# Pad the sequences using the pad_sequences() method
X_pad = 

### Create the Training, Validation, and Testing Sets

You need to create suitable training, validation, and testing sets for fitting and testing the RNN LSTM model using the encoded review comments. Use the `train_test_split` method from `sklearn` to create these sets.

In [ ]:
# Creating training, validation, and testing sets using the encoded data
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = 

X_train_rnn, X_val_rnn, y_train_rnn, y_val_rnn = 

### Build and Train the RNN LSTM Model

Remember that we use `Embedding` layers to analyze text data in RNN LSTM models, so this section starts by setting-up some initial variables needed for the RNN LSTM to score sentiment.

As it's defined in the [Embedding layer documentation of the Keras API](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding?version=stable), you should set the `input_dim` parameter to the size of your vocabulary, so we set the `vocabulary_size` variable to the length of the number of words in the `tokenizer` plus `1`.

Also, we define a variable called `embedding_size` to specify how many dimensions will be used to represent each word.

In [ ]:
# Import Keras modules for model creation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
# Model set-up
vocabulary_size = len(tokenizer.word_counts.keys()) + 1
embedding_size = 64

Define an RNN LSTM model as follows:

* _Layer 1:_ Add an `Embedding` layer using the `vocabulary_size` and `embedding_size` variables as the first two parameters, and setting `input_length=max_words` (the same size as the padding).

* _Layer 2:_ Add an LSTM layer with `280` units.

* _Output Layer:_ Add a `Dense` layer with `1` unit and `sigmoid` as activation function.

In [ ]:
# Define the LSTM RNN model
model = Sequential()

# Layer 1

# Layer 2

# Output layer


Compile the model using the `binary_crossentropy` loss function, the `adam` optimizer, and fetch the following metrics: Accuracy, True positives, True negatives, False positives, False negatives, Precision, Recall, and AUC.

In [ ]:
# Compile the model


Display the summary of the model using the `summary` method of the model.

In [ ]:
# Show model summary


Train the RNN LSTM model using a batch size equals to `1000` and ten epochs. Remember to set the `validation_data` parameter to use your validation sets.

In [ ]:
# Training the model


Use the `predict_classes` method of your model to score the sentiment setting `batch_size=1000`.

In [ ]:
# Predict classes using the testing data
y_rnn_pred = 

## Models Comparison

In this section, you will assess the performance of VADER and the RNN LSTM to score sentiment to decide which one is better.

### Accuracy

Use the `accuracy_score` method from `sklearn` to calculate the accuracy of each model. Display the results for further analysis.

In [ ]:
# Accuracy
from sklearn.metrics import accuracy_score


### Confusion Matrix

Scoring the sentiment of the movie reviews as positive or negative is a binary classification problem, so use the `confusion_matrix` method from `sklearn` to calculate the confusion matrix for VADER and the RNN LSTM model.

In [ ]:
# Import the confusion_matrix method from sklearn
from sklearn.metrics import confusion_matrix

#### Confusion matrix for VADER

Create the confusion matrix for vader passing the `y_test` and `y_vader_pred` variables as parameters.

In [ ]:
# Confusion matrtix metrics from Vader
tn_vader, fp_vader, fn_vader, tp_vader = 

# Dataframe to display confusion matrix from Vader
cm_vader_df = pd.DataFrame(
    {
        "Positive(1)": [f"TP={tp_vader}", f"FP={fp_vader}"],
        "Negative(0)": [f"FN={fn_vader}", f"TN={tn_vader}"],
    },
    index=["Positive(1)", "Negative(0)"],
)
cm_vader_df.index.name = "Actual"
cm_vader_df.columns.name = "Predicted"
print("Confusion Matrix from Vader")
display(cm_vader_df)

#### Confusion matrix for the RNN LSTM Model

Create the confusion matrix for the RNN LSTM model passing the `y_test_rnn` and `y_rnn_pred` variables as parameters.

In [ ]:
# Confusion matrtix metrics from the RNN LSTM model
tn_rnn, fp_rnn, fn_rnn, tp_rnn = 

# Dataframe to display confusion matrix from the RNN LSTM model
cm_rnn_df = pd.DataFrame(
    {
        "Positive(1)": [f"TP={tp_rnn}", f"FP={fp_rnn}"],
        "Negative(0)": [f"FN={fn_rnn}", f"TN={tn_rnn}"],
    },
    index=["Positive(1)", "Negative(0)"],
)
cm_rnn_df.index.name = "Actual"
cm_rnn_df.columns.name = "Predicted"
print("Confusion Matrix from the RNN LSTM Model")
display(cm_rnn_df)

### Classification Report

Use the `classification_report` from `sklearn` and generate a report for each model.

In [ ]:
# Import the classification_report method from sklearn
from sklearn.metrics import classification_report

In [ ]:
# Display classification report for Vader
print("Classification Report for Vader")

In [ ]:
# Display classification report for the RNN LSTM Model
print("Classification Report for the RNN LSTM Model")

### Plotting the ROC Curve

In this section, you will visually analyze the performance of both models by plotting the ROC Curve. You will use the `roc_curve` and `auc` methods from `sklearn` to gather the data needed to plot this curve.

In [ ]:
# Import the roc_curve and auc metrics from sklearn
from sklearn.metrics import roc_curve, auc

#### ROC Curve - VADER

Use the `roc_curve` method from `sklearn` to calculate the false positives (`fpr`) and true positives (`tpr`) rates passing as parameters the testing target sentiments (`y_test`) and the normalized values of `y_vader_prob` (e.g. `y_vader_prob_norm`).

In [ ]:
# Data for ROC Curve - VADER
fpr_test_vader, tpr_test_vader, thresholds_test_vader = 

After calculating the `fpr` and `tpr` for VADER, use the `auc` method of `sklearn` to calculate the AUC for VADER. Round the final result up to `4` decimals.

In [ ]:
# AUC for VADER
auc_test_vader =

Once you gather all the data needed to plot the ROC curve, create a DataFrame with the `fpr` and `tpr` data from VADER.

In [ ]:
# Dataframe to plot ROC Curve for VADER
roc_df_test_vader = 

Using the `plot()` method of the DataFrame, plot the ROC Curve in red color and show the AUC value in the plot title.

_Hint:_ You can pass `xlim=([-0.05, 1.05])` as a parameter to the `plot()` method to better adjust the curve.

#### ROC Curve RNN LSTM

Use the `predict()` method from the RNN LSTM model to predict the sentiment of the testing data `X_test_rnn`. Set `batch_size=1000` to speed up the predictions and store the results in a variable called `test_predictions_rnn`.

In [ ]:
# Making predictions to feed the roc_curve module
test_predictions_rnn = 

Use the `roc_curve` method from `sklearn` to calculate the false positives (`fpr`) and true positives (`tpr`) rates passing as parameters the testing target sentiments (`y_test_rnn`) and the predictions you compute using the testing data (`test_predictions_rnn`).

In [ ]:
# Data for ROC Curve - RNN LSTM Model
fpr_test_rnn, tpr_test_rnn, thresholds_test_rnn = 

After calculating the `fpr` and `tpr` for the RNN LSTM Model, use the `auc` method of `sklearn` to calculate the AUC for this model. Round the final result up to `4` decimals.

In [ ]:
# AUC for the RNN LSTM Model
auc_test_rnn = 

Once you gather all the data needed to plot the ROC curve, create a DataFrame with the `fpr` and `tpr` data from the RNN LSTM model.

In [ ]:
# Dataframe to plot ROC Curve for the RNN LSTM model
roc_df_test_rnn = 

Using the `plot()` method of the DataFrame, plot the ROC Curve in blue color and show the AUC value in the plot title.

_Hint:_ You can pass `xlim=([-0.05, 1.05])` as a parameter to the `plot()` method to better adjust the curve.

## Results Analysis and Conclusions

Review all the metrics you computed, evaluate the ROC curve plots, and the AUC values to answer the following question:

* Which model performed best scoring sentiments?

    **Your answer here**